In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7f3f2041dc40>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
interval = '30m'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 48
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = True

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5381 entries, 2023-08-16 00:00:00 to 2023-12-06 02:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     5381 non-null   datetime64[ns]
 1   open                          5381 non-null   float32       
 2   high                          5381 non-null   float32       
 3   low                           5381 non-null   float32       
 4   close                         5381 non-null   float32       
 5   volume                        5381 non-null   float32       
 6   close_time                    5381 non-null   int64         
 7   quote_asset_volume            5381 non-null   float32       
 8   number_of_trades              5381 non-null   int16         
 9   taker_buy_base_asset_volume   5381 non-null   float32       
 10  taker_buy_quote_asset_volume  5381 non-null   float32       

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2023-08-16 00:00:00,2023-08-16 00:00:00,29200.009766,29237.269531,29196.210938,29216.179688,197.675812,1692145799999,5775957.5,8500,107.259773,3133953.25,BTCUSDT
2023-08-16 00:30:00,2023-08-16 00:30:00,29216.189453,29259.849609,29199.990234,29224.369141,441.255524,1692147599999,12894129.0,11314,203.166153,5937846.00,BTCUSDT
2023-08-16 01:00:00,2023-08-16 01:00:00,29224.369141,29240.000000,29209.960938,29221.000000,337.218231,1692149399999,9854230.0,9187,148.748917,4346948.00,BTCUSDT
2023-08-16 01:30:00,2023-08-16 01:30:00,29221.000000,29249.019531,29184.960938,29245.470703,618.671753,1692151199999,18076154.0,11472,266.341156,7781488.50,BTCUSDT
2023-08-16 02:00:00,2023-08-16 02:00:00,29245.460938,29257.460938,29228.800781,29228.800781,794.825989,1692152999999,23243630.0,11308,391.669800,11453557.00,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 00:00:00,2023-12-06 00:00:00,44073.820312,44104.000000,43919.980469,43942.789062,1474.109253,1701822599999,64885536.0,-30704,670.200867,29497944.00,BTCUSDT
2023-12-06 00:30:00,2023-12-06 00:30:00,43942.800781,44146.078125,43830.449219,44139.000000,1276.207031,1701824399999,56117824.0,30510,740.101501,32551480.00,BTCUSDT
2023-12-06 01:00:00,2023-12-06 01:00:00,44139.000000,44145.371094,43955.449219,43958.371094,919.661621,1701826199999,40506260.0,-28280,442.733459,19500512.00,BTCUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5381 entries, 2023-08-16 00:00:00 to 2023-12-06 02:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     5381 non-null   datetime64[ns]
 1   open                          5381 non-null   float32       
 2   high                          5381 non-null   float32       
 3   low                           5381 non-null   float32       
 4   close                         5381 non-null   float32       
 5   volume                        5381 non-null   float32       
 6   close_time                    5381 non-null   int64         
 7   quote_asset_volume            5381 non-null   float32       
 8   number_of_trades              5381 non-null   int16         
 9   taker_buy_base_asset_volume   5381 non-null   float32       
 10  taker_buy_quote_asset_volume  5381 non-null   float32       

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2023-08-16 00:00:00,2023-08-16 00:00:00,29200.009766,29237.269531,29196.210938,29216.179688,197.675812,1692145799999,5775957.5,8500,107.259773,3133953.25,BTCUSDT,NaN
2023-08-16 00:30:00,2023-08-16 00:30:00,29216.189453,29259.849609,29199.990234,29224.369141,441.255524,1692147599999,12894129.0,11314,203.166153,5937846.00,BTCUSDT,NaN
2023-08-16 01:00:00,2023-08-16 01:00:00,29224.369141,29240.000000,29209.960938,29221.000000,337.218231,1692149399999,9854230.0,9187,148.748917,4346948.00,BTCUSDT,NaN
2023-08-16 01:30:00,2023-08-16 01:30:00,29221.000000,29249.019531,29184.960938,29245.470703,618.671753,1692151199999,18076154.0,11472,266.341156,7781488.50,BTCUSDT,NaN
2023-08-16 02:00:00,2023-08-16 02:00:00,29245.460938,29257.460938,29228.800781,29228.800781,794.825989,1692152999999,23243630.0,11308,391.669800,11453557.00,BTCUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 00:00:00,2023-12-06 00:00:00,44073.820312,44104.000000,43919.980469,43942.789062,1474.109253,1701822599999,64885536.0,-30704,670.200867,29497944.00,BTCUSDT,64.256577
2023-12-06 00:30:00,2023-12-06 00:30:00,43942.800781,44146.078125,43830.449219,44139.000000,1276.207031,1701824399999,56117824.0,30510,740.101501,32551480.00,BTCUSDT,67.008774
2023-12-06 01:00:00,2023-12-06 01:00:00,44139.000000,44145.371094,43955.449219,43958.371094,919.661621,1701826199999,40506260.0,-28280,442.733459,19500512.00,BTCUSDT,62.256344


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5381 entries, 2023-08-16 00:00:00 to 2023-12-06 02:00:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     5381 non-null   datetime64[ns]
 1   open                          5381 non-null   float32       
 2   high                          5381 non-null   float32       
 3   low                           5381 non-null   float32       
 4   close                         5381 non-null   float32       
 5   volume                        5381 non-null   float32       
 6   close_time                    5381 non-null   int64         
 7   quote_asset_volume            5381 non-null   float32       
 8   number_of_trades              5381 non-null   int16         
 9   taker_buy_base_asset_volume   5381 non-null   float32       
 10  taker_buy_quote_asset_volume  5381 non-null   float32       

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_48p,ema_48p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2023-08-16 00:00:00,2023-08-16 00:00:00,29200.009766,29237.269531,29196.210938,29216.179688,197.675812,1692145799999,5775957.5,8500,107.259773,3133953.25,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-16 00:30:00,2023-08-16 00:30:00,29216.189453,29259.849609,29199.990234,29224.369141,441.255524,1692147599999,12894129.0,11314,203.166153,5937846.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-16 01:00:00,2023-08-16 01:00:00,29224.369141,29240.000000,29209.960938,29221.000000,337.218231,1692149399999,9854230.0,9187,148.748917,4346948.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-16 01:30:00,2023-08-16 01:30:00,29221.000000,29249.019531,29184.960938,29245.470703,618.671753,1692151199999,18076154.0,11472,266.341156,7781488.50,BTCUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-16 02:00:00,2023-08-16 02:00:00,29245.460938,29257.460938,29228.800781,29228.800781,794.825989,1692152999999,23243630.0,11308,391.669800,11453557.00,BTCUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 00:00:00,2023-12-06 00:00:00,44073.820312,44104.000000,43919.980469,43942.789062,1474.109253,1701822599999,64885536.0,-30704,670.200867,29497944.00,BTCUSDT,64.256577,42764.812500,2.754546,40903.222656,7.431118
2023-12-06 00:30:00,2023-12-06 00:30:00,43942.800781,44146.078125,43830.449219,44139.000000,1276.207031,1701824399999,56117824.0,30510,740.101501,32551480.00,BTCUSDT,67.008774,42820.902344,3.078164,40935.421875,7.825932
2023-12-06 01:00:00,2023-12-06 01:00:00,44139.000000,44145.371094,43955.449219,43958.371094,919.661621,1701826199999,40506260.0,-28280,442.733459,19500512.00,BTCUSDT,62.256344,42867.332031,2.545153,40965.500000,7.305833


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5182 entries, 2023-08-20 03:30:00 to 2023-12-06 02:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     5182 non-null   datetime64[ns]
 1   open                          5182 non-null   float32       
 2   high                          5182 non-null   float32       
 3   low                           5182 non-null   float32       
 4   close                         5182 non-null   float32       
 5   volume                        5182 non-null   float32       
 6   close_time                    5182 non-null   int64         
 7   quote_asset_volume            5182 non-null   float32       
 8   number_of_trades              5182 non-null   int16         
 9   taker_buy_base_asset_volume   5182 non-null   float32       
 10  taker_buy_quote_asset_volume  5182 non-null   float32       

In [7]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5182 entries, 2023-08-20 03:30:00 to 2023-12-06 02:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     5182 non-null   datetime64[ns]
 1   open                          5182 non-null   float32       
 2   high                          5182 non-null   float32       
 3   low                           5182 non-null   float32       
 4   close                         5182 non-null   float32       
 5   volume                        5182 non-null   float32       
 6   close_time                    5182 non-null   int64         
 7   quote_asset_volume            5182 non-null   float32       
 8   number_of_trades              5182 non-null   int16         
 9   taker_buy_base_asset_volume   5182 non-null   float32       
 10  taker_buy_quote_asset_volume  5182 non-null   float32       

In [8]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
ESTAVEL,1636,0.315708
LONG_1.0,2074,0.400232
SHORT_1.0,1472,0.284060


In [9]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_48p,ema_48p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,


In [10]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_48p,ema_48p_diff,ema_200p,ema_200p_diff,status
ix_open_time,,,,,,,,,,,,,,,,,,
2023-08-20 03:30:00,2023-08-20 03:30:00,26130.919922,26143.039062,26103.460938,26105.240234,430.855621,1692503999999,11254781.0,8867,286.199036,7476055.50,BTCUSDT,52.145691,26099.255859,0.022929,26715.296875,-2.283548,ESTAVEL
2023-08-20 04:00:00,2023-08-20 04:00:00,26105.230469,26125.910156,26105.230469,26115.220703,407.423676,1692505799999,10639987.0,7870,298.275177,7789494.00,BTCUSDT,53.137405,26099.908203,0.058669,26708.402344,-2.220955,ESTAVEL
2023-08-20 04:30:00,2023-08-20 04:30:00,26115.210938,26131.210938,26101.250000,26121.470703,442.855194,1692507599999,11564594.0,7126,287.977722,7519945.00,BTCUSDT,53.783321,26100.789062,0.079238,26701.667969,-2.172888,ESTAVEL
2023-08-20 05:00:00,2023-08-20 05:00:00,26121.460938,26160.160156,26110.769531,26147.119141,623.259888,1692509399999,16284904.0,8708,349.471954,9131389.00,BTCUSDT,56.436920,26102.679688,0.170249,26695.316406,-2.053534,ESTAVEL
2023-08-20 05:30:00,2023-08-20 05:30:00,26147.109375,26147.119141,26067.699219,26092.000000,269.995087,1692511199999,7047956.0,8387,130.287170,3400505.25,BTCUSDT,49.817173,26102.244141,-0.039246,26688.416016,-2.234737,ESTAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 00:00:00,2023-12-06 00:00:00,44073.820312,44104.000000,43919.980469,43942.789062,1474.109253,1701822599999,64885536.0,-30704,670.200867,29497944.00,BTCUSDT,64.256577,42764.812500,2.754546,40903.222656,7.431118,ESTAVEL
2023-12-06 00:30:00,2023-12-06 00:30:00,43942.800781,44146.078125,43830.449219,44139.000000,1276.207031,1701824399999,56117824.0,30510,740.101501,32551480.00,BTCUSDT,67.008774,42820.902344,3.078164,40935.421875,7.825932,ESTAVEL
2023-12-06 01:00:00,2023-12-06 01:00:00,44139.000000,44145.371094,43955.449219,43958.371094,919.661621,1701826199999,40506260.0,-28280,442.733459,19500512.00,BTCUSDT,62.256344,42867.332031,2.545153,40965.500000,7.305833,ESTAVEL


In [11]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
text_numeric_features = f'number_of_trades,quote_asset_volume,ema_{int(times_regression_PnL)}p,ema_200p'
numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['number_of_trades', 'quote_asset_volume', 'ema_48p', 'ema_200p'] - size: 4

Combination Numeric Features: 
['number_of_trades', 'quote_asset_volume', 'ema_48p', 'ema_200p', 'number_of_trades,quote_asset_volume', 'number_of_trades,ema_48p', 'number_of_trades,ema_200p', 'quote_asset_volume,ema_48p', 'quote_asset_volume,ema_200p', 'ema_48p,ema_200p', 'number_of_trades,quote_asset_volume,ema_48p', 'number_of_trades,quote_asset_volume,ema_200p', 'number_of_trades,ema_48p,ema_200p', 'quote_asset_volume,ema_48p,ema_200p', 'number_of_trades,quote_asset_volume,ema_48p,ema_200p']


In [12]:
from datetime import datetime
import os
from imblearn.under_sampling import InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, AllKNN


#iht = InstanceHardnessThreshold(estimator=KNeighborsClassifier())

# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = 'repeatededitednearestneighbours,instancehardnessthreshold,allknn'.split(',')
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:
    if has_simulation_file:
      chave = (df_resultado_simulacao['symbol'] == symbol) & \
          (df_resultado_simulacao['interval'] == interval) & \
          (df_resultado_simulacao['estimator'] == estimator) & \
          (df_resultado_simulacao['fix_imbalance_method'] == fix_imbalance_method) & \
          (df_resultado_simulacao['features'] == str(features))

      if chave.sum() > 0:
        print(f'fix_imbalance_method: {fix_imbalance_method} already exists')
        continue

    try:
      print(f'fix_imbalance_method: {fix_imbalance_method}')
      fim = None
      match(fix_imbalance_method):
        case 'repeatededitednearestneighbours':
          fim = RepeatedEditedNearestNeighbours(kind_sel='all', max_iter=100, n_jobs=20, n_neighbors=3, sampling_strategy='auto')
        case 'instancehardnessthreshold':
          fim = InstanceHardnessThreshold(cv=5, estimator=None, n_jobs=20, random_state=123, sampling_strategy='auto')
        case 'allknn':
          fim = AllKNN(allow_minority=False, kind_sel='all', n_jobs=20, n_neighbors=3, sampling_strategy='auto')

      setup = experiement.setup(
          data=train_data[features].copy(),
          train_size=myenv.train_size,
          target=myenv.label,
          numeric_features=aux_numeric_features.split(','),
          date_features=['open_time'],
          create_date_columns=["hour", "day", "month"],
          data_split_shuffle=False,
          data_split_stratify=False,
          fix_imbalance=True,
          fix_imbalance_method=fim,
          remove_outliers=True,
          fold_strategy='timeseries', 
          fold=3,
          session_id=123,
          normalize=normalize,
          use_gpu=True,
          verbose=True,
          n_jobs=20,
          log_experiment=False,
      )

      if _compare_models:
        best = setup.compare_models()
        estimator = setup.pull().index[0]
        print('Estimator: ' + estimator)
      else:
        best = setup.create_model(estimator, n_jobs=20)

      if tune_model:
        best = setup.tune_model(best)

      # predict on test set
      # holdout_pred = setup.predict_model(best)
      # print('Holdout Score:', holdout_pred['prediction_score'].mean())
      # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

      predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
      predict[myenv.label] = test_data[myenv.label]
      predict['_score'] = predict['prediction_label'] == predict[myenv.label]
      # print('Predict Score Mean:', predict['_score'].mean())
      # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

      final_model = setup.finalize_model(best)

      ajusted_test_data = test_data.drop(myenv.label, axis=1)
      df_final_predict, res_score = utils.validate_score_test_data(
          setup,
          final_model,
          myenv.label,
          test_data,
          ajusted_test_data)

      # df_final_predict.info()
      # print('Final Score Mean:', res_score.mean().values[0])
      # print('Final Score Group:\n', res_score)

      start_test_date = df_final_predict['open_time'].min()
      end_test_date = df_final_predict['open_time'].max()

      # print('Simule Trading:')
      # print(f'Min Data: {start_test_date}')
      # print(f'Max Data: {end_test_date}')
      saldo_inicial = 100.0
      saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
      print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

      result_simulado = {}
      result_simulado['date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      result_simulado['symbol'] = symbol
      result_simulado['interval'] = interval
      result_simulado['estimator'] = estimator
      result_simulado['fix_imbalance_method'] = fix_imbalance_method
      result_simulado['stop_loss'] = stop_loss
      result_simulado['times_regression_profit_and_loss'] = times_regression_PnL
      result_simulado['features'] = features
      result_simulado['final_value'] = saldo_final

      if res_score is not None:
        result_simulado['score'] = ''
        for i in range(0, len(res_score.index.values)):
          result_simulado['score'] += f'[{res_score.index.values[i]}={res_score["_score"].values[i]:.2f}]'

      df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
      df_resultado_simulacao.sort_values('final_value', inplace=True)

      df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)
    except Exception as e:
      print(e)
      continue

features: ['open_time', 'status', 'number_of_trades']
fix_imbalance_method: repeatededitednearestneighbours
With n_samples=0, test_size=None and train_size=0.7, the resulting train set will be empty. Adjust any of the aforementioned parameters.
fix_imbalance_method: instancehardnessthreshold
With n_samples=0, test_size=None and train_size=0.7, the resulting train set will be empty. Adjust any of the aforementioned parameters.
fix_imbalance_method: allknn
With n_samples=0, test_size=None and train_size=0.7, the resulting train set will be empty. Adjust any of the aforementioned parameters.
features: ['open_time', 'status', 'quote_asset_volume']
fix_imbalance_method: repeatededitednearestneighbours
With n_samples=0, test_size=None and train_size=0.7, the resulting train set will be empty. Adjust any of the aforementioned parameters.
fix_imbalance_method: instancehardnessthreshold
With n_samples=0, test_size=None and train_size=0.7, the resulting train set will be empty. Adjust any of the

In [13]:
final_model

NameError: name 'final_model' is not defined